In [32]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

import re

from collections import Counter

In [40]:
input_data = 'train_final.csv'
output_data = 'train_final_tokenized.csv'

In [41]:
df = pd.read_csv(input_data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11544 entries, 0 to 11543
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        11544 non-null  int64 
 1   Category  11544 non-null  int64 
 2   Sentence  11544 non-null  object
dtypes: int64(2), object(1)
memory usage: 270.7+ KB


In [42]:
tokencol = []
stop_words = set(stopwords.words('english'))
punct = {'.', ',', '?', '!', '@', '#', '$', '%', '^', '&', '*', '(', ')', '-', '_', '-lrb-', '-rrb-', '`', '...', "'", ';', ':', '--', '``', }
for Sentence in df['Sentence']:
    result = []
    token = word_tokenize(Sentence.lower())
    for word in token:
        if word not in stop_words and word not in punct:
            result.append(word)
    tokencol.append(result)
df['Token'] = tokencol

In [43]:
df.head()

,Id,Category,Sentence,Token
0,0,3,-LRB- The film -RRB- tackles the topic of rela...,"[film, tackles, topic, relationships, straight..."
1,1,2,"Lavishly , exhilaratingly tasteless .","[lavishly, exhilaratingly, tasteless]"
2,2,4,It is also beautifully acted .,"[also, beautifully, acted]"
3,3,1,"But , like Silence , it 's a movie that gets u...","[like, silence, 's, movie, gets, skin]"
4,4,2,It 's been made with an innocent yet fervid co...,"['s, made, innocent, yet, fervid, conviction, ..."


In [14]:
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [15]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
tokenized = df['Sentence'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
df['BERTToken']=tokenized

In [45]:
df.head()

,Id,Category,Sentence,Token,BERTToken
0,0,3,-LRB- The film -RRB- tackles the topic of rela...,"[film, tackles, topic, relationships, straight...","[101, 1011, 1048, 15185, 1011, 1996, 2143, 101..."
1,1,2,"Lavishly , exhilaratingly tasteless .","[lavishly, exhilaratingly, tasteless]","[101, 22689, 2135, 1010, 4654, 26415, 15172, 2..."
2,2,4,It is also beautifully acted .,"[also, beautifully, acted]","[101, 2009, 2003, 2036, 17950, 6051, 1012, 102]"
3,3,1,"But , like Silence , it 's a movie that gets u...","[like, silence, 's, movie, gets, skin]","[101, 2021, 1010, 2066, 4223, 1010, 2009, 1005..."
4,4,2,It 's been made with an innocent yet fervid co...,"['s, made, innocent, yet, fervid, conviction, ...","[101, 2009, 1005, 1055, 2042, 2081, 2007, 2019..."


In [46]:
df.to_csv(output_data, index=False)